$\newcommand{\set}[2]{\big\{#1\,\ {\large:}\ \,#2\big\}}
\newcommand{\eps}{\varepsilon}
\newcommand{\abs}[1]{\left\vert#1\right\vert}
\newcommand{\ceil}[1]{\left\lceil#1\right\rceil}
\newcommand{\floor}[1]{\left\lfloor#1\right\rfloor}
$
# Busca incremental e busca adaptativa

### $ 2. 2 $ Descrição do procedimento para localização de uma raiz e critérios de parada

Todos os métodos para localização de zeros que estudaremos são *iterativos*. Partindo de uma estimativa inicial para um zero $ \zeta $, a cada passo utilizamos a estimativa anterior para obter uma aproximação mais refinada para $ \zeta $, até que esta seja julgada boa o suficiente. Os *critérios de parada* mais comuns são:
1. A distância entre o zero $ \zeta $ e sua aproximação atual é menor que um $ \eps > 0 $ escolhido previamente.
2. O valor da função na aproximação atual é menor que $ \eps $ em valor absoluto.
3. O número de iterações excede uma cota $ N$ pré-fixada.

**Problema 4:** Construa um procedimento `checa_zero` que tem por parâmetros: uma função real $ f $; dois pontos $ a $ e $ b $ em seu domínio; e uma constante $ \eps > 0 $; e que retorna `True` se e somente se:
* $ \abs{b-a} < \eps $; e
* $ \abs{f(m)} < \eps $, onde $ m $ é o ponto médio de $ a $ e $ b $.

*Solução:*

### $ 2.3 $ Encaixotamento de zeros

Qualquer dos métodos que estudaremos requer como passo preliminar o **encaixotamento** de um zero, ou seja, a determinação de um intervalo onde $ f $ troca de sinal. A escolha deste intervalo é crucial: para algumas escolhas o método em questão pode convergir muito lentamente ou até falhar.

Para encaixotar um zero de uma função, as três opções mais comuns são:
* Usar a a teoria subjacente para advinhar a sua localização aproximada, no caso em que a função provém de um modelo da Física ou Engenharia;
* Esboçar o gráfico da função e estimar visualmente um subintervalo onde ele cruza o eixo-$x$;
* Aplicar uma busca sistemática, avaliando o sinal da função em pontos sucessivos para localizar um subintervalo onde a função troca de sinal.

Destes três métodos, apenas o terceiro é rígido o suficiente para ser programado com facilidade, o que não quer dizer que os outros dois sejam menos valiosos.

📝 **Isolar** um zero significa encontrar um intervalo que o contém em seu interior mas que não contém qualquer outro zero. Se um intervalo contém mais de um zero, em geral não há como controlar para qual deles um método iterativo convergirá, por isto sempre que possível é desejável isolar um zero, não somente encaixotá-lo.

## $ \S 4 $ Busca incremental para encaixotamento de raízes

Suponha que queiramos encaixotar um zero de uma função contínua $ f \colon [a, b] \to \mathbb{R} $. Usando o método da **busca incremental**, começamos escolhendo um **incremento** ou **tamanho de passo** $ h > 0 $. Então calculamos o sinal de
$$
f(x_i)f(x_{i+1}) \quad \text{para} \quad x_i = a+ih \quad (i = 0, 1, \dots )
$$
sucessivamente.
* Se $ f(x_i)f(x_{i+1}) \le 0 $ para algum $ i $, então o intervalo $ [x_i,x_{i+1}] $ deve conter um zero de $ f $, pelo Corolário 2.
* Caso contrário, eventualmente teremos $ x_{i+1} > b $ e a busca terá sido inconclusiva.

In [ ]:
def busca_incremental(f, a, b, h):
    """
    Começando com x_1 = a e x_2 = a + h e com incrementos
    de h, retorna o primeiro par de pontos consecutivos onde
    f assume sinais opostos.
    """
    from numpy import sign
    
    # Inicializando:
    x_0 = a
    x_1 = a + h
    f_0 = f(x_0)
    f_1 = f(x_1)
    
    while sign(f_0) == sign(f_1):
        if x_1 > b:
            return None, None
        x_0, f_0 = x_1, f_1
        x_1 += h
        f_1 = f(x_1)
        
    return x_0, x_1

**Problema 5:** O polinômio $ x^3 - 11x^2 + 7 $ possui exatamente um zero entre $ 0 $ e $ 1 $. Encaixote este zero dentro de um intervalo de comprimento no máximo $ 10^{-3} $.

*Solução:*

⚠️ Se $ f $ não tem zeros no intervalo $ [a,b] $, obviamente a busca incremental será mal-sucedida. Por outro lado, mesmo que $ f $ tenha zeros aí, por menor que seja o valor do incremento $ h $, não há como garantir *a priori* que a busca será bem sucedida, ou que o intervalo resultante conterá um *único* zero.

⚠️ Muitas vezes precisamos encontrar *todos* os zeros de uma função. Nestes casos, se utilizarmos um incremento $ h $ grande demais, corremos o risco de pular por um número par de zeros consecutivos sem detectá-los, ou de obter um intervalo que contém um número ímpar de zeros em seu interior, mas ao final conseguir localizar apenas um deles. Por outro lado, se $ h $ for pequeno demais, gastaremos muito tempo procurando em regiões que não contêm qualquer zero.

**Problema 6:**

(a) Quantos zeros a função $ f(x) = \sin(x) - 0.999 $ tem no intervalo $ [0, 30\pi] $?

(b) Mostre que `busca_incremental` com tamanho de passo $ h = 1 $ não consegue encaixotar nenhum deles.

*Solução:*

In [ ]:
from numpy import sin, pi

## $ \S 5 $ Busca incremental adaptativa

**Definição:** As funções

\begin{alignat*}{9}
    \floor{\cdot} \colon \mathbb{R} \to \mathbb{Z},
    \quad \floor{x} & = \text{maior inteiro $ \le x $} \\
    \ceil{\cdot} \colon \mathbb{R} \to \mathbb{Z},
    \quad \ceil{x} & = \text{menor inteiro $ \ge x $} \\
\end{alignat*}
são chamadas de funções **chão** e **teto** respectivamente. 

Observe que estas funções são contínuas exceto nos inteiros, onde têm uma descontinuidade de salto.

**Problema 7:** Mostre que
$$
\ceil{x} =
\begin{cases}
    \floor{x} + 1 & \text{se $ x \not \in \mathbb{Z} $} \\
    \floor{x} & \text{se $ x \in \mathbb{Z} $}
\end{cases}
$$



**Teorema 2:** _No pior caso, a busca incremental aplicada a uma função definida no intervalo $ [a, b] $ com tamanho de passo $ h $ requer_
$$
    \boxed{n = \floor{\frac{b-a}{h}} + 1 \ \text{avaliações}.}
$$

**Prova:** Suponha que a função à qual a busca foi aplicada não possua zeros em $ [a, b] $. Então precisamos avaliá-la em cada ponto $ x_i = a + ih $ tal que $ x_i \le b $, começando de $ i = 0 $. O último inteiro $ i $ que satisfaz esta condição é
$$
    \floor{\frac{b-a}{h}}. \tag*{$\blacksquare$}
$$

Por exemplo, se quisermos localizar o zero de uma função definida no intervalo $ [0, 1] $ com precisão de ao menos $ 5 $ dígitos, precisamos realizar a princípio $ 10^5 $ avaliações da função. Assim, para uma tolerância pequena relativamente ao comprimento do intervalo original, o custo computacional pode ser inaceitavelmente alto.

Uma alternativa para tentar reduzir este custo é implementar uma **busca adaptativa**, em que começamos com um incremento de tamanho $ h = \frac{b-a}{2} $ e a cada passo reduzimos o seu comprimento pela metade até que ele fique menor que a tolerância desejada. O ponto crucial é que a cada vez que encontrarmos um subintervalo onde a função troca de sinal, podemos restringir nossa busca a este intervalo menor para diminuir o número de avaliações.

In [ ]:
def busca_adaptativa(f, a, b, h_max):
    """
    Começando com incremento de tamanho h = (b - a) / 2,
    a cada passo realiza uma busca incremental no intervalo [a, b]
    anterior com tamanho de passo h / 2. Se a busca é bem sucedida,
    atualizamos a, b às saídas x_0, x_1; caso contrário, utilizamos
    os mesmos valores de a, b no próximo passo. A busca termina assim
    que h < h_max.
    """
    from numpy import sign
    
    # Inicializando:
    x_0 = a
    x_1 = b
    h =  (b - a) / 2
    
    while h >= h_max:
        x_0, x_1 = busca_incremental(f, a, b, h)
        if x_0:
            a = x_0
            b = x_1
        h /= 2
    
    if x_0:
        return x_0, x_1
    else:
        print("Busca inconclusiva!")
        return None, None

⚠️ Não há garantia que a busca adaptativa seja sempre mais eficiente que a busca incremental simples.

**Problema:** Modifique `busca_incremental` para criar um novo procedimento `busca_exaustiva` com os mesmos parâmetros que antes, mas que retorna uma lista contendo todos os pares $ \big(x_{i-1}, x_i\big) $ tais que ocorre uma troca de sinal no intervalo com estas extremidades, onde
$$
    x_i = a + h i \quad \text{para} \quad i = 0,1,\dots,N - 1 =\floor{\frac{b-a}{h}} \quad\text{e} \quad x_N = b.
$$

*Solução:*